In [ ]:
#beginning, Team6

In [1]:
%matplotlib inline
import matplotlib
import os              # This provides several system utilities
import pandas as pd    # This is the workhorse of data munging in Python
import numpy as np     # This is for general numerical operations 
import seaborn as sns  # This allows us to efficiently and beautifully plot
import networkx as nx  
import osmnx as ox
import statsmodels.api as sm
import matplotlib.pyplot as plt

# display settings (optional)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

In [2]:
#for the import process, next step is to create a string variable containing the directory of the file and a file name variable
infodir = r'C:\Users\EPRO\OneDrive - Wageningen University & Research\Course Channel Metropolitan Challenges 2025 - Paper Group 6 Parking space vs Food Space'
fname = infodir + r'\data\buurten_filtered.csv'

gpdf01 = pd.read_csv(fname)
#this is to verify
gpdf01.head()

,OBJECTNUMMER,BRT,SDC,Periode,Dagdeel,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT
0,117.0,AA01 Planciusbuurt-Noord,A,202403-202404,doordeweeks avond,55.0,52.0,3.0,0.0,55.0,95.0,100.0,"POLYGON((4.8857454 52.3881665,4.8844789 52.388...","POLYGON((52.3881665 4.8857454,52.3883785 4.884...",4.884390,52.387023
1,118.0,AA02 Westelijke eilanden,A,202403-202404,doordeweeks avond,313.0,287.0,9.0,0.0,296.0,92.0,95.0,"POLYGON((4.8910092 52.3884398,4.8859714 52.388...","POLYGON((52.3884398 4.8910092,52.3881287 4.885...",4.887548,52.385401
2,119.0,AA03 Westerdokseiland,A,202403-202404,doordeweeks avond,165.0,76.0,0.0,0.0,76.0,46.0,46.0,"POLYGON((4.8950776 52.3886909,4.8923612 52.388...","POLYGON((52.3886909 4.8950776,52.3885233 4.892...",4.894474,52.384417
3,120.0,AA04 Planciusbuurt-Zuid,A,202403-202404,doordeweeks avond,20.0,15.0,1.0,0.0,16.0,75.0,80.0,"POLYGON((4.8839232 52.385505,4.884072 52.38566...","POLYGON((52.385505 4.8839232,52.3856666 4.8840...",4.882867,52.384947
4,121.0,AA05 Haarlemmerbuurt-West,A,202403-202404,doordeweeks avond,148.0,141.0,4.0,0.0,145.0,95.0,98.0,"POLYGON((4.8839232 52.385505,4.8835118 52.3849...","POLYGON((52.385505 4.8839232,52.3849538 4.8835...",4.886377,52.383013


In [3]:
#Team6 is dropping all entries that do not have "doordeweeks avond" in "Dagdeel"
#Team6 chose to focus on the most likely time cars will be at home after a day at work
gpdf01_1 = gpdf01[gpdf01['Dagdeel'] == 'doordeweeks avond']
#Team6 is creating column Wijken, to merge more easily with future databases for maps
gpdf01_1['Wijken'] = gpdf01_1['BRT'].str[:2]
#team6 is creating a column with the buurt code to keep it
gpdf01_1['BRT_code'] = gpdf01_1['BRT'].astype(str).str[:4]
#team6 is separating the name of each neighborhood to merge more easily with future databases
gpdf01_1['BRT_name'] = gpdf01_1['BRT'].str[5:]
#team6 is replacing all dashes in the name of each neighborhood to merge more easily with future databases
gpdf01_1['BRT_name'] = gpdf01_1['BRT_name'].str.replace('-', ' ', regex=False)
gpdf01_1.head(10)

C:\Users\EPRO\AppData\Local\Temp\ipykernel_1984\2552515082.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpdf01_1['Wijken'] = gpdf01_1['BRT'].str[:2]
C:\Users\EPRO\AppData\Local\Temp\ipykernel_1984\2552515082.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpdf01_1['BRT_code'] = gpdf01_1['BRT'].astype(str).str[:4]
C:\Users\EPRO\AppData\Local\Temp\ipykernel_1984\2552515082.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,OBJECTNUMMER,BRT,SDC,Periode,Dagdeel,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Wijken,BRT_code,BRT_name
0,117.0,AA01 Planciusbuurt-Noord,A,202403-202404,doordeweeks avond,55.0,52.0,3.0,0.0,55.0,95.0,100.0,"POLYGON((4.8857454 52.3881665,4.8844789 52.388...","POLYGON((52.3881665 4.8857454,52.3883785 4.884...",4.884390,52.387023,AA,AA01,Planciusbuurt Noord
1,118.0,AA02 Westelijke eilanden,A,202403-202404,doordeweeks avond,313.0,287.0,9.0,0.0,296.0,92.0,95.0,"POLYGON((4.8910092 52.3884398,4.8859714 52.388...","POLYGON((52.3884398 4.8910092,52.3881287 4.885...",4.887548,52.385401,AA,AA02,Westelijke eilanden
2,119.0,AA03 Westerdokseiland,A,202403-202404,doordeweeks avond,165.0,76.0,0.0,0.0,76.0,46.0,46.0,"POLYGON((4.8950776 52.3886909,4.8923612 52.388...","POLYGON((52.3886909 4.8950776,52.3885233 4.892...",4.894474,52.384417,AA,AA03,Westerdokseiland
3,120.0,AA04 Planciusbuurt-Zuid,A,202403-202404,doordeweeks avond,20.0,15.0,1.0,0.0,16.0,75.0,80.0,"POLYGON((4.8839232 52.385505,4.884072 52.38566...","POLYGON((52.385505 4.8839232,52.3856666 4.8840...",4.882867,52.384947,AA,AA04,Planciusbuurt Zuid
4,121.0,AA05 Haarlemmerbuurt-West,A,202403-202404,doordeweeks avond,148.0,141.0,4.0,0.0,145.0,95.0,98.0,"POLYGON((4.8839232 52.385505,4.8835118 52.3849...","POLYGON((52.385505 4.8839232,52.3849538 4.8835...",4.886377,52.383013,AA,AA05,Haarlemmerbuurt West
5,122.0,AA06 Haarlemmerbuurt-Oost,A,202403-202404,doordeweeks avond,121.0,110.0,1.0,0.0,111.0,91.0,92.0,"POLYGON((4.8901799 52.3823939,4.8901504 52.382...","POLYGON((52.3823939 4.8901799,52.3823625 4.890...",4.892116,52.380579,AA,AA06,Haarlemmerbuurt Oost
6,123.0,AB01 Marnixbuurt-Noord,A,202403-202404,doordeweeks avond,169.0,137.0,1.0,0.0,138.0,81.0,82.0,"POLYGON((4.8829591 52.3836506,4.8829307 52.383...","POLYGON((52.3836506 4.8829591,52.3837005 4.882...",4.880560,52.380942,AB,AB01,Marnixbuurt Noord
7,124.0,AB02 Driehoekbuurt,A,202403-202404,doordeweeks avond,312.0,266.0,3.0,0.0,269.0,85.0,86.0,"POLYGON((4.8829591 52.3836506,4.8803612 52.379...","POLYGON((52.3836506 4.8829591,52.37958 4.88036...",4.884213,52.381615,AB,AB02,Driehoekbuurt
8,125.0,AB03 Anjeliersbuurt-Noord,A,202403-202404,doordeweeks avond,165.0,141.0,0.0,0.0,141.0,85.0,85.0,"POLYGON((4.8871229 52.3790942,4.8881601 52.380...","POLYGON((52.3790942 4.8871229,52.3802151 4.888...",4.883804,52.379197,AB,AB03,Anjeliersbuurt Noord
9,126.0,AB04 Zaagpoortbuurt,A,202403-202404,doordeweeks avond,158.0,85.0,2.0,0.0,87.0,54.0,55.0,"POLYGON((4.8791773 52.3777046,4.8786863 52.377...","POLYGON((52.3777046 4.8791773,52.3778162 4.878...",4.877343,52.376228,AB,AB04,Zaagpoortbuurt


In [4]:
#team6 is checking how many we have left
before = len(gpdf01)
after = len(gpdf01_1)
print(f"Rows before: {before}, after: {after}, dropped: {before - after}")

Rows before: 1447, after: 518, dropped: 929


In [28]:
#team6 is purging all rows with "netto_pkd" and "bruto_pkd" higher than 150% because that looks like missing and damaged data
#that would otherwise interfere with our next process
gpdf01_2 = gpdf01_1[(gpdf01_1['Netto_PKD'] <= 150) & (gpdf01_1['Bruto_PKD'] <= 150)]
gpdf01_2.head(10)

,OBJECTNUMMER,BRT,SDC,Periode,Dagdeel,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Wijken,BRT_code,BRT_name
157,563.0,EG02 GWL-terrein,E,202409-202410,doordeweeks avond,38.0,39.0,0.0,0.0,39.0,103.0,103.0,"POLYGON((4.8694074 52.3855428,4.8668659 52.385...","POLYGON((52.3855428 4.8694074,52.3854968 4.866...",4.868767,52.383811,EG,EG02,GWL terrein
70,357.0,TA03 Amstel III deel A/B-Noord,T,202406,doordeweeks avond,4.0,4.0,0.0,0.0,4.0,100.0,100.0,"POLYGON((4.9475058 52.3108069,4.9463557 52.312...","POLYGON((52.3108069 4.9475058,52.3120613 4.946...",4.946693,52.305855,TA,TA03,Amstel III deel A/B Noord
89,376.0,TF02 Grunder/Koningshoef,T,202406,doordeweeks avond,425.0,416.0,24.0,0.0,440.0,98.0,104.0,"POLYGON((4.9769028 52.3238067,4.977699 52.3229...","POLYGON((52.3238067 4.9769028,52.3229333 4.977...",4.980608,52.321355,TF,TF02,Grunder/Koningshoef
171,577.0,EK03 Jan Maijenbuurt,E,202409-202410,doordeweeks avond,260.0,251.0,0.0,0.0,251.0,97.0,97.0,"POLYGON((4.8524111 52.3733998,4.8511174 52.373...","POLYGON((52.3733998 4.8524111,52.3731711 4.851...",4.851281,52.371718,EK,EK03,Jan Maijenbuurt
466,1542.0,KK04 Scheldebuurt-Oost,K,202502-202503,doordeweeks avond,512.0,497.0,5.0,0.0,502.0,97.0,98.0,"POLYGON((4.9052324 52.3462901,4.9039292 52.346...","POLYGON((52.3462901 4.9052324,52.3461078 4.903...",4.903181,52.344702,KK,KK04,Scheldebuurt Oost
139,545.0,EB08 Zeeheldenbuurt,E,202409-202410,doordeweeks avond,212.0,205.0,7.0,0.0,212.0,97.0,100.0,"POLYGON((4.8856593 52.3907556,4.8855179 52.390...","POLYGON((52.3907556 4.8856593,52.3904709 4.885...",4.888207,52.389442,EB,EB08,Zeeheldenbuurt
143,549.0,EC02 Houthavens-Oost,E,202409-202410,doordeweeks avond,49.0,47.0,0.0,0.0,47.0,96.0,96.0,"POLYGON((4.8915542 52.3951121,4.8910925 52.394...","POLYGON((52.3951121 4.8915542,52.3948517 4.891...",4.890368,52.391651,EC,EC02,Houthavens Oost
444,1520.0,KF01 Cornelis Troostbuurt,K,202502-202503,doordeweeks avond,607.0,580.0,1.0,0.0,581.0,96.0,96.0,"POLYGON((4.8911303 52.3527902,4.8870604 52.352...","POLYGON((52.3527902 4.8911303,52.3522456 4.887...",4.888934,52.350508,KF,KF01,Cornelis Troostbuurt
281,1032.0,FB07 Bakemabuurt,F,202409-202411,doordeweeks avond,533.0,513.0,24.0,0.0,537.0,96.0,101.0,"POLYGON((4.8093999 52.3793915,4.8094205 52.379...","POLYGON((52.3793915 4.8093999,52.379517 4.8094...",4.805575,52.378331,FB,FB07,Bakemabuurt
87,374.0,TE03 G-buurt-Noord,T,202406,doordeweeks avond,320.0,306.0,29.0,0.0,335.0,96.0,105.0,"POLYGON((4.9714808 52.3253526,4.9706196 52.325...","POLYGON((52.3253526 4.9714808,52.3250574 4.970...",4.970181,52.323912,TE,TE03,G buurt Noord


In [29]:
gpdf01_2 = gpdf01_2.sort_values(by=["Netto_PKD", "Bruto_PKD"], ascending=[False, True])
gpdf01_2.head(10)

,OBJECTNUMMER,BRT,SDC,Periode,Dagdeel,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Wijken,BRT_code,BRT_name
157,563.0,EG02 GWL-terrein,E,202409-202410,doordeweeks avond,38.0,39.0,0.0,0.0,39.0,103.0,103.0,"POLYGON((4.8694074 52.3855428,4.8668659 52.385...","POLYGON((52.3855428 4.8694074,52.3854968 4.866...",4.868767,52.383811,EG,EG02,GWL terrein
70,357.0,TA03 Amstel III deel A/B-Noord,T,202406,doordeweeks avond,4.0,4.0,0.0,0.0,4.0,100.0,100.0,"POLYGON((4.9475058 52.3108069,4.9463557 52.312...","POLYGON((52.3108069 4.9475058,52.3120613 4.946...",4.946693,52.305855,TA,TA03,Amstel III deel A/B Noord
89,376.0,TF02 Grunder/Koningshoef,T,202406,doordeweeks avond,425.0,416.0,24.0,0.0,440.0,98.0,104.0,"POLYGON((4.9769028 52.3238067,4.977699 52.3229...","POLYGON((52.3238067 4.9769028,52.3229333 4.977...",4.980608,52.321355,TF,TF02,Grunder/Koningshoef
171,577.0,EK03 Jan Maijenbuurt,E,202409-202410,doordeweeks avond,260.0,251.0,0.0,0.0,251.0,97.0,97.0,"POLYGON((4.8524111 52.3733998,4.8511174 52.373...","POLYGON((52.3733998 4.8524111,52.3731711 4.851...",4.851281,52.371718,EK,EK03,Jan Maijenbuurt
466,1542.0,KK04 Scheldebuurt-Oost,K,202502-202503,doordeweeks avond,512.0,497.0,5.0,0.0,502.0,97.0,98.0,"POLYGON((4.9052324 52.3462901,4.9039292 52.346...","POLYGON((52.3462901 4.9052324,52.3461078 4.903...",4.903181,52.344702,KK,KK04,Scheldebuurt Oost
139,545.0,EB08 Zeeheldenbuurt,E,202409-202410,doordeweeks avond,212.0,205.0,7.0,0.0,212.0,97.0,100.0,"POLYGON((4.8856593 52.3907556,4.8855179 52.390...","POLYGON((52.3907556 4.8856593,52.3904709 4.885...",4.888207,52.389442,EB,EB08,Zeeheldenbuurt
143,549.0,EC02 Houthavens-Oost,E,202409-202410,doordeweeks avond,49.0,47.0,0.0,0.0,47.0,96.0,96.0,"POLYGON((4.8915542 52.3951121,4.8910925 52.394...","POLYGON((52.3951121 4.8915542,52.3948517 4.891...",4.890368,52.391651,EC,EC02,Houthavens Oost
444,1520.0,KF01 Cornelis Troostbuurt,K,202502-202503,doordeweeks avond,607.0,580.0,1.0,0.0,581.0,96.0,96.0,"POLYGON((4.8911303 52.3527902,4.8870604 52.352...","POLYGON((52.3527902 4.8911303,52.3522456 4.887...",4.888934,52.350508,KF,KF01,Cornelis Troostbuurt
281,1032.0,FB07 Bakemabuurt,F,202409-202411,doordeweeks avond,533.0,513.0,24.0,0.0,537.0,96.0,101.0,"POLYGON((4.8093999 52.3793915,4.8094205 52.379...","POLYGON((52.3793915 4.8093999,52.379517 4.8094...",4.805575,52.378331,FB,FB07,Bakemabuurt
87,374.0,TE03 G-buurt-Noord,T,202406,doordeweeks avond,320.0,306.0,29.0,0.0,335.0,96.0,105.0,"POLYGON((4.9714808 52.3253526,4.9706196 52.325...","POLYGON((52.3253526 4.9714808,52.3250574 4.970...",4.970181,52.323912,TE,TE03,G buurt Noord


In [30]:
#team6 slicing database to focus on the main usefull content
gpdf01_2 = gpdf01_2.drop(columns=['BRT', 'Dagdeel', 'WKT_LNG_LAT', 'WKT_LAT_LNG', 'LNG', 'LAT'])
#ordering the database columns to be easier to read
gpdf01_2 = gpdf01_2[['OBJECTNUMMER', 'Wijken', 'BRT_code', 'BRT_name'] + [col for col in gpdf01_2.columns if col not in ['OBJECTNUMMER', 'Wijken', 'BRT_code', 'BRT_name']]]
gpdf01_2.head()

,OBJECTNUMMER,Wijken,BRT_code,BRT_name,SDC,Periode,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD
157,563.0,EG,EG02,GWL terrein,E,202409-202410,38.0,39.0,0.0,0.0,39.0,103.0,103.0
70,357.0,TA,TA03,Amstel III deel A/B Noord,T,202406,4.0,4.0,0.0,0.0,4.0,100.0,100.0
89,376.0,TF,TF02,Grunder/Koningshoef,T,202406,425.0,416.0,24.0,0.0,440.0,98.0,104.0
171,577.0,EK,EK03,Jan Maijenbuurt,E,202409-202410,260.0,251.0,0.0,0.0,251.0,97.0,97.0
466,1542.0,KK,KK04,Scheldebuurt Oost,K,202502-202503,512.0,497.0,5.0,0.0,502.0,97.0,98.0


In [7]:
#for the import process, next step is to create a string variable containing the directory of the file and a file name variable
infodir2 = r'C:\Users\EPRO\OneDrive - Wageningen University & Research\Course Channel Metropolitan Challenges 2025 - Paper Group 6 Parking space vs Food Space'
fname2 = infodir2 + r'\data\KEA_db_green_and_gray_ngbhd_2023_03_trnsltd_only_AMS.csv'

gpdf02g = pd.read_csv(fname2)
#this is to verify
gpdf02g.head()

,OBJID,mnpty_code,mnpty_name,ngbhd_code,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,qty_inhbt,perc_built_area,perc_water_total,m2_water,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,perc_publcnt,m2_publc_green,perc_public_green,perc_publc_tree,perc_trnsn_total,perc_publcnt_tree,perc_publcnt_green,perc_publcnt_greennt,perc_other,...,m2_built_envmt,m2_built_envmt_greennt,m2_green_other,m2_other_surfc,m2_other_greennt,m2_tree_publc,m2_tree_publcnt,m2_tree_built_envmt,m2_tree_water,perc_green,perc_low_green,perc_tree,perc_built_gray,m2_agrct_green,m2_agrct,m2_agrct_greennt,m2_trnsn_greennt,m2_green_trnsn,m2_trnsn,m2_agrct_tree,m2_tree_other,m2_tree_trnsn,perc_trnsn_tree,perc_trnsn_green,perc_trnsn_greennt
0,7,GM0363,Amsterdam,BU03631306,4427,Westergasfabriek,1,-99999999,0,99.999999,12.47,26504.03,255419.18,7.28,99685.39,68.27,10.31,44776.51,17.53,29.24,1.67,1.06,0.43,8.82,0.0,...,18382.65,18382.65,0.0,4.67,4.67,74683.12,2719.53,199.06,5333.44,50.53,18.01,32.52,39.09,0.0,0.0,0.0,4022.37,130.2,4152.57,0.0,0.0,130.49,0.23,3.28,3.60
1,45,GM0363,Amsterdam,BU03634709,4552,Duivelseiland,1,23729,1385,100.000003,4.97,2920.45,61646.98,34.69,23080.23,42.51,17.82,202.57,0.33,5.07,0.00,5.94,0.69,11.19,0.0,...,20876.45,20876.45,0.0,0.00,0.00,3126.23,3662.17,514.10,141.05,13.09,1.02,12.07,82.16,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.46,4.27,8.62
2,53,GM0363,Amsterdam,BU03634600,4539,Valeriusbuurt Oost,1,12248,1405,99.999996,4.55,4607.40,120651.50,32.67,38367.50,40.32,22.34,2521.47,2.09,8.52,0.12,8.73,1.51,12.10,0.0,...,38507.74,38507.74,0.0,0.00,0.00,10281.06,10538.49,906.86,879.44,22.33,3.60,18.73,73.85,0.0,0.0,0.0,144.76,0.0,144.76,0.0,0.0,0.00,1.25,1.37,1.83
3,90,GM0363,Amsterdam,BU03635301,4579,IJselbuurt Oost,1,21880,2900,99.999988,16.80,26146.56,159417.36,25.28,54516.32,40.28,17.55,2078.39,1.30,6.09,0.10,5.97,2.77,8.81,0.0,...,39400.79,39378.99,0.0,0.00,0.00,9703.89,9513.20,901.93,631.04,17.14,4.09,13.05,66.47,0.0,0.0,0.0,109.68,8.4,118.08,0.0,0.0,32.23,0.03,0.68,9.48
4,92,GM0363,Amsterdam,BU03639303,4774,F-buurt,1,20169,4465,100.000004,6.38,14141.01,233166.41,24.11,121287.55,56.32,13.12,12907.19,5.54,4.30,0.06,0.43,0.42,12.27,0.0,...,56080.41,56080.41,0.0,0.00,0.00,10036.16,1006.84,141.33,735.41,11.07,5.96,5.11,82.86,0.0,0.0,0.0,145.69,0.0,145.69,0.0,0.0,0.00,0.00,0.01,16.96


In [8]:
#team6 slicing database to focus on the main usefull content
gpdf02g_1 = gpdf02g.drop(columns=['OBJID', 'ngbhd_code', 'perc_other', 'perc_publcnt', 'perc_trnsn_total', 'perc_publcnt_tree',
                                      'perc_publcnt_green', 'perc_publcnt_greennt', 'perc_built_tree', 'perc_built_green', 'perc_built_greennt',
                                      'perc_other_tree', 'perc_other_green', 'perc_other_greennt', 'perc_trnsn_tree', 'perc_trnsn_green',
                                      'perc_trnsn_greennt', 'perc_water', 'perc_water_tree', 'm2_ngbhd_pls_agrct', 'm2_publicnt_green',
                                      'm2_publicnt', 'm2_publcnt_greennt', 'm2_publicnt_green', 'm2_publicnt', 'm2_publcnt_greennt',
                                      'm2_built_envmt_green', 'm2_built_envmt', 'm2_built_envmt_greennt', 'm2_water', 'm2_agrct_green', 'm2_agrct',
                                      'm2_agrct_greennt', 'm2_green_other', 'm2_other_surfc', 'm2_other_greennt', 'm2_green_trnsn', 'm2_trnsn',
                                      'm2_trnsn_greennt', 'm2_tree_publc', 'm2_tree_publcnt', 'm2_tree_built_envmt', 'm2_tree_water', 'm2_agrct_tree',
                                      'm2_tree_other', 'm2_tree_trnsn', 'perc_built_gray', 'mnpty_code', 'mnpty_name', 'qty_inhbt', 'perc_built_area'])
gpdf02g_1.head()

,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
0,4427,Westergasfabriek,1,-99999999,12.47,255419.18,7.28,99685.39,68.27,44776.51,17.53,29.24,21.50,54908.88,50.53,18.01,32.52
1,4552,Duivelseiland,1,23729,4.97,61646.98,34.69,23080.23,42.51,202.57,0.33,5.07,37.11,22877.66,13.09,1.02,12.07
2,4539,Valeriusbuurt Oost,1,12248,4.55,120651.50,32.67,38367.50,40.32,2521.47,2.09,8.52,29.71,35846.03,22.33,3.60,18.73
3,4579,IJselbuurt Oost,1,21880,16.80,159417.36,25.28,54516.32,40.28,2078.39,1.30,6.09,32.89,52437.93,17.14,4.09,13.05
4,4774,F-buurt,1,20169,6.38,233166.41,24.11,121287.55,56.32,12907.19,5.54,4.30,46.48,108380.36,11.07,5.96,5.11


In [9]:
#looking for any cells with no values on them
gpdf02g_1[(gpdf02g_1 == '').any(axis=1)]

,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree


In [10]:
#team6 replacing all -99999999 values in pop_density with 0 just to interfere less with our analysis
gpdf02g_1['pop_dnsty_km2'] = gpdf02g_1['pop_dnsty_km2'].replace(-99999999, 0)
gpdf02g_1.head(10)

,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
0,4427,Westergasfabriek,1,0,12.47,255419.18,7.28,99685.39,68.27,44776.51,17.53,29.24,21.50,54908.88,50.53,18.01,32.52
1,4552,Duivelseiland,1,23729,4.97,61646.98,34.69,23080.23,42.51,202.57,0.33,5.07,37.11,22877.66,13.09,1.02,12.07
2,4539,Valeriusbuurt Oost,1,12248,4.55,120651.50,32.67,38367.50,40.32,2521.47,2.09,8.52,29.71,35846.03,22.33,3.60,18.73
3,4579,IJselbuurt Oost,1,21880,16.80,159417.36,25.28,54516.32,40.28,2078.39,1.30,6.09,32.89,52437.93,17.14,4.09,13.05
4,4774,F-buurt,1,20169,6.38,233166.41,24.11,121287.55,56.32,12907.19,5.54,4.30,46.48,108380.36,11.07,5.96,5.11
5,4578,IJselbuurt West,1,19329,7.15,133355.16,26.85,47469.49,50.09,3845.38,2.88,14.50,32.71,43624.12,26.81,5.42,21.39
6,4787,Kelbergen,1,4980,3.54,218062.29,11.15,107361.42,71.03,14376.39,6.59,21.80,42.64,92985.03,33.53,9.08,24.45
7,4509,Bedrijventerrein Sloterdijk I,3,129,0.71,682174.55,30.53,254587.23,45.45,55362.94,8.12,8.13,29.20,199224.29,17.45,8.58,8.87
8,4390,Sarphatistrook,1,11904,15.20,216363.28,33.15,66800.83,39.32,2647.74,1.22,8.45,29.65,64153.08,14.91,1.82,13.09
9,4391,de Plantage,1,5285,11.54,378109.04,17.37,63442.67,25.60,5470.57,1.45,8.82,15.33,57972.09,33.41,3.96,29.45


In [11]:
#team6 preparing the neighborhood names to fit more easily with the other data set
gpdf02g_1['ngbhd_name'] = gpdf02g_1['ngbhd_name'].str.replace("-", " ", regex=False)
gpdf02g_1.head(10)

,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
0,4427,Westergasfabriek,1,0,12.47,255419.18,7.28,99685.39,68.27,44776.51,17.53,29.24,21.50,54908.88,50.53,18.01,32.52
1,4552,Duivelseiland,1,23729,4.97,61646.98,34.69,23080.23,42.51,202.57,0.33,5.07,37.11,22877.66,13.09,1.02,12.07
2,4539,Valeriusbuurt Oost,1,12248,4.55,120651.50,32.67,38367.50,40.32,2521.47,2.09,8.52,29.71,35846.03,22.33,3.60,18.73
3,4579,IJselbuurt Oost,1,21880,16.80,159417.36,25.28,54516.32,40.28,2078.39,1.30,6.09,32.89,52437.93,17.14,4.09,13.05
4,4774,F buurt,1,20169,6.38,233166.41,24.11,121287.55,56.32,12907.19,5.54,4.30,46.48,108380.36,11.07,5.96,5.11
5,4578,IJselbuurt West,1,19329,7.15,133355.16,26.85,47469.49,50.09,3845.38,2.88,14.50,32.71,43624.12,26.81,5.42,21.39
6,4787,Kelbergen,1,4980,3.54,218062.29,11.15,107361.42,71.03,14376.39,6.59,21.80,42.64,92985.03,33.53,9.08,24.45
7,4509,Bedrijventerrein Sloterdijk I,3,129,0.71,682174.55,30.53,254587.23,45.45,55362.94,8.12,8.13,29.20,199224.29,17.45,8.58,8.87
8,4390,Sarphatistrook,1,11904,15.20,216363.28,33.15,66800.83,39.32,2647.74,1.22,8.45,29.65,64153.08,14.91,1.82,13.09
9,4391,de Plantage,1,5285,11.54,378109.04,17.37,63442.67,25.60,5470.57,1.45,8.82,15.33,57972.09,33.41,3.96,29.45


In [31]:
#team6 needs to look for matches to ease the process in merging both data bases
matches = set(gpdf01_2['BRT_name']) & set(gpdf02g_1['ngbhd_name'])
num_matches = len(matches)
#to count the number of non-matching values:
num_nomatch = len(set(gpdf01_2['BRT_name']) | set(gpdf02g_1['ngbhd_name'])) - num_matches

print(f'Number of matching values: {num_matches}')
print(f'Number of non-matching values: {num_nomatch}')

Number of matching values: 385
Number of non-matching values: 199


In [32]:
#merging both databases as outer, to keep the non matched items
gpdf03mf = pd.merge(gpdf01_2, gpdf02g_1, left_on="BRT_name", right_on="ngbhd_name", how="outer")
gpdf03mf.head(10)

,OBJECTNUMMER,Wijken,BRT_code,BRT_name,SDC,Periode,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
0,360.0,TA,TA06,AMC,T,202406,21.0,3.0,1.0,0.0,4.0,14.0,19.0,4769.0,AMC,3.0,43.0,6.50,801628.18,12.78,329669.54,63.60,42373.40,5.29,22.47,35.84,287296.14,36.67,7.35,29.32
1,1498.0,KA,KA05,Aalsmeerwegbuurt Oost,K,202502-202503,545.0,480.0,0.0,0.0,480.0,88.0,88.0,4534.0,Aalsmeerwegbuurt Oost,1.0,19568.0,4.90,118059.55,28.40,43482.84,47.06,2846.04,2.41,10.23,34.42,40636.80,20.11,4.68,15.43
2,1497.0,KA,KA04,Aalsmeerwegbuurt West,K,202502-202503,521.0,473.0,1.0,0.0,474.0,91.0,91.0,4294.0,Aalsmeerwegbuurt West,1.0,25911.0,6.96,105240.24,31.06,31308.95,42.08,1773.90,1.69,12.33,28.06,29535.05,23.06,3.88,19.18
3,1786.0,SE,SE03,Aetsveld Noord,S,202503,766.0,588.0,27.0,0.0,615.0,77.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1785.0,SE,SE02,Aetsveld Zuid,S,202503,700.0,571.0,27.0,0.0,598.0,82.0,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1784.0,SE,SE01,Aetsveldsepolder,S,202503,5.0,4.0,0.0,0.0,4.0,80.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,492.0,BA,BA01,Afrikahaven,B,202406,38.0,10.0,0.0,0.0,10.0,26.0,26.0,4410.0,Afrikahaven,5.0,0.0,23.35,5217434.25,7.62,701613.19,14.27,222692.00,4.27,0.82,9.18,478921.19,25.16,19.76,5.40
7,172.0,AJ,AJ04,Alexanderplein e.o.,A,202403-202404,88.0,66.0,0.0,0.0,66.0,75.0,75.0,1322.0,Alexanderplein e.o.,1.0,9784.0,18.22,48868.23,21.25,15823.40,46.24,3148.78,6.44,13.86,25.94,12674.62,28.02,7.01,21.01
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4402.0,Alfa driehoek,5.0,0.0,4.97,411778.23,11.63,182485.38,52.08,53973.64,13.11,7.76,31.21,128511.74,23.23,14.39,8.84
9,1295.0,MF,MF03,Ambonpleinbuurt,M,202501-202503,667.0,559.0,4.0,0.0,563.0,84.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
gpdf03mf_a = gpdf03mf.sort_values(by=["Netto_PKD", "perc_publc_space"], ascending=[False, True])
gpdf03mf_a.head(10)

,OBJECTNUMMER,Wijken,BRT_code,BRT_name,SDC,Periode,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
189,563.0,EG,EG02,GWL terrein,E,202409-202410,38.0,39.0,0.0,0.0,39.0,103.0,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,357.0,TA,TA03,Amstel III deel A/B Noord,T,202406,4.0,4.0,0.0,0.0,4.0,100.0,100.0,4764.0,Amstel III deel A/B Noord,2.0,97.0,7.81,800491.96,21.57,389875.81,53.47,35637.61,4.45,4.77,44.25,354238.20,11.49,5.16,6.33
210,376.0,TF,TF02,Grunder/Koningshoef,T,202406,425.0,416.0,24.0,0.0,440.0,98.0,104.0,4793.0,Grunder/Koningshoef,1.0,14500.0,8.65,131388.59,25.10,57838.37,48.60,4731.01,3.60,4.58,40.42,53107.36,9.41,4.66,4.75
255,577.0,EK,EK03,Jan Maijenbuurt,E,202409-202410,260.0,251.0,0.0,0.0,251.0,97.0,97.0,4527.0,Jan Maijenbuurt,1.0,29359.0,0.00,78222.90,36.31,25461.52,39.48,1009.53,1.29,6.93,31.26,24451.99,18.84,4.49,14.35
575,545.0,EB,EB08,Zeeheldenbuurt,E,202409-202410,212.0,205.0,7.0,0.0,212.0,97.0,100.0,4422.0,Zeeheldenbuurt,1.0,22444.0,12.65,103570.96,30.98,37724.84,41.64,410.40,0.40,5.21,36.03,37314.44,11.54,2.91,8.63
444,1542.0,KK,KK04,Scheldebuurt Oost,K,202502-202503,512.0,497.0,5.0,0.0,502.0,97.0,98.0,4575.0,Scheldebuurt Oost,1.0,16775.0,0.00,140439.17,24.95,51886.30,51.37,5780.15,4.12,14.42,32.83,46106.16,28.17,6.13,22.04
244,549.0,EC,EC02,Houthavens Oost,E,202409-202410,49.0,47.0,0.0,0.0,47.0,96.0,96.0,4421.0,Houthavens Oost,1.0,28896.0,92.56,256234.18,1.42,14891.43,5.88,694.90,0.27,0.07,5.54,14196.53,0.40,0.27,0.13
118,1520.0,KF,KF01,Cornelis Troostbuurt,K,202502-202503,607.0,580.0,1.0,0.0,581.0,96.0,96.0,4467.0,Cornelis Troostbuurt,1.0,23871.0,6.25,156682.00,36.32,47809.58,37.21,618.15,0.39,6.70,30.12,47191.43,18.99,2.73,16.26
186,374.0,TE,TE03,G buurt Noord,T,202406,320.0,306.0,29.0,0.0,335.0,96.0,105.0,4799.0,G buurt Noord,1.0,22316.0,8.27,81241.17,10.73,44959.04,76.75,12430.82,15.30,21.41,40.04,32528.22,38.71,15.77,22.94
33,1032.0,FB,FB07,Bakemabuurt,F,202409-202411,533.0,513.0,24.0,0.0,537.0,96.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
gpdf03mf_d = gpdf03mf.sort_values(by=["Netto_PKD", "perc_publc_space"], ascending=[True, False])
gpdf03mf_d.head(10)

,OBJECTNUMMER,Wijken,BRT_code,BRT_name,SDC,Periode,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
367,1272.0,MA,MA03,Oostelijke Handelskade,M,202501-202503,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4474.0,Oostelijke Handelskade,1.0,6381.0,39.35,364079.79,12.24,103670.81,29.06,7904.12,2.17,0.59,26.30,95766.69,3.07,2.22,0.85
477,1330.0,MN,MN09,Sportpark Voorland,M,202501-202503,204.0,2.0,0.0,0.0,2.0,1.0,1.0,4600.0,Sportpark Voorland,2.0,0.0,6.51,269864.58,0.01,72910.47,30.35,14060.37,5.21,3.33,21.81,58850.10,35.14,17.31,17.83
377,1302.0,MH,MH03,Oostpunt Zeeburgereiland,M,202501-202503,97.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,1311.0,MK,MK02,Pampusbuurt West,M,202501-202503,229.0,2.0,0.0,0.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,834.0,NQ,NQ02,Noorderstrook Oost,N,202410-202411,396.0,13.0,2.0,0.0,15.0,3.0,4.0,4680.0,Noorderstrook Oost,2.0,0.0,14.91,480031.74,0.46,118130.97,43.20,60712.97,12.65,18.59,11.96,57418.00,57.29,29.13,28.16
336,500.0,BB,BB03,Minervahaven Noordoost,B,202406,62.0,2.0,0.0,0.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,1300.0,MH,MH01,Sluisbuurt,M,202501-202503,37.0,1.0,1.0,0.0,2.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,1567.0,KR,KR04,Amstelpark,K,202502-202503,405.0,17.0,1.0,0.0,18.0,4.0,4.0,4762.0,Amstelpark,2.0,24.0,21.59,622328.41,0.87,175408.23,72.63,73081.75,11.74,44.45,16.44,102326.48,64.02,12.97,51.05
21,1558.0,KQ,KQ01,Amsterdamse Bos,K,202502-202503,130.0,5.0,1.0,0.0,6.0,4.0,5.0,4754.0,Amsterdamse Bos,4.0,90.0,51.97,1111505.74,3.14,209079.26,40.77,120275.47,10.82,21.96,7.99,88803.79,37.67,11.47,26.20
193,391.0,TK,TK03,Gaasperplas,T,202406,224.0,8.0,0.0,0.0,8.0,4.0,4.0,4803.0,Gaasperplas,5.0,48.0,50.07,1193317.09,3.11,275916.74,38.07,133471.07,11.18,14.95,11.94,142445.67,31.81,13.82,17.99


In [33]:
#merging both databases as outer, to keep the non matched items
gpdf03mp = pd.merge(gpdf01_2, gpdf02g_1, left_on="BRT_name", right_on="ngbhd_name", how="inner")
gpdf03mp.head(10)

,OBJECTNUMMER,Wijken,BRT_code,BRT_name,SDC,Periode,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
0,357.0,TA,TA03,Amstel III deel A/B Noord,T,202406,4.0,4.0,0.0,0.0,4.0,100.0,100.0,4764,Amstel III deel A/B Noord,2,97,7.81,800491.96,21.57,389875.81,53.47,35637.61,4.45,4.77,44.25,354238.20,11.49,5.16,6.33
1,376.0,TF,TF02,Grunder/Koningshoef,T,202406,425.0,416.0,24.0,0.0,440.0,98.0,104.0,4793,Grunder/Koningshoef,1,14500,8.65,131388.59,25.10,57838.37,48.60,4731.01,3.60,4.58,40.42,53107.36,9.41,4.66,4.75
2,577.0,EK,EK03,Jan Maijenbuurt,E,202409-202410,260.0,251.0,0.0,0.0,251.0,97.0,97.0,4527,Jan Maijenbuurt,1,29359,0.00,78222.90,36.31,25461.52,39.48,1009.53,1.29,6.93,31.26,24451.99,18.84,4.49,14.35
3,1542.0,KK,KK04,Scheldebuurt Oost,K,202502-202503,512.0,497.0,5.0,0.0,502.0,97.0,98.0,4575,Scheldebuurt Oost,1,16775,0.00,140439.17,24.95,51886.30,51.37,5780.15,4.12,14.42,32.83,46106.16,28.17,6.13,22.04
4,545.0,EB,EB08,Zeeheldenbuurt,E,202409-202410,212.0,205.0,7.0,0.0,212.0,97.0,100.0,4422,Zeeheldenbuurt,1,22444,12.65,103570.96,30.98,37724.84,41.64,410.40,0.40,5.21,36.03,37314.44,11.54,2.91,8.63
5,549.0,EC,EC02,Houthavens Oost,E,202409-202410,49.0,47.0,0.0,0.0,47.0,96.0,96.0,4421,Houthavens Oost,1,28896,92.56,256234.18,1.42,14891.43,5.88,694.90,0.27,0.07,5.54,14196.53,0.40,0.27,0.13
6,1520.0,KF,KF01,Cornelis Troostbuurt,K,202502-202503,607.0,580.0,1.0,0.0,581.0,96.0,96.0,4467,Cornelis Troostbuurt,1,23871,6.25,156682.00,36.32,47809.58,37.21,618.15,0.39,6.70,30.12,47191.43,18.99,2.73,16.26
7,374.0,TE,TE03,G buurt Noord,T,202406,320.0,306.0,29.0,0.0,335.0,96.0,105.0,4799,G buurt Noord,1,22316,8.27,81241.17,10.73,44959.04,76.75,12430.82,15.30,21.41,40.04,32528.22,38.71,15.77,22.94
8,180.0,AK,AK08,Kazernebuurt,A,202403-202404,103.0,98.0,0.0,0.0,98.0,95.0,95.0,4398,Kazernebuurt,1,17470,17.35,66600.93,27.41,22955.19,49.41,3054.68,4.59,14.94,29.88,19900.51,25.20,6.08,19.12
9,1521.0,KF,KF02,Lizzy Ansinghbuurt,K,202502-202503,239.0,226.0,2.0,0.0,228.0,95.0,95.0,4466,Lizzy Ansinghbuurt,1,19122,3.08,127548.67,40.61,33939.18,32.37,540.94,0.42,5.77,26.18,33398.24,17.84,2.51,15.33


In [47]:
gpdf03mp_a = gpdf03mp.sort_values(by=["Netto_PKD", "perc_public_green"], ascending=[False, True])
gpdf03mp_a.head(10)

,OBJECTNUMMER,Wijken,BRT_code,BRT_name,SDC,Periode,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
0,357.0,TA,TA03,Amstel III deel A/B Noord,T,202406,4.0,4.0,0.0,0.0,4.0,100.0,100.0,4764,Amstel III deel A/B Noord,2,97,7.81,800491.96,21.57,389875.81,53.47,35637.61,4.45,4.77,44.25,354238.20,11.49,5.16,6.33
1,376.0,TF,TF02,Grunder/Koningshoef,T,202406,425.0,416.0,24.0,0.0,440.0,98.0,104.0,4793,Grunder/Koningshoef,1,14500,8.65,131388.59,25.10,57838.37,48.60,4731.01,3.60,4.58,40.42,53107.36,9.41,4.66,4.75
4,545.0,EB,EB08,Zeeheldenbuurt,E,202409-202410,212.0,205.0,7.0,0.0,212.0,97.0,100.0,4422,Zeeheldenbuurt,1,22444,12.65,103570.96,30.98,37724.84,41.64,410.40,0.40,5.21,36.03,37314.44,11.54,2.91,8.63
2,577.0,EK,EK03,Jan Maijenbuurt,E,202409-202410,260.0,251.0,0.0,0.0,251.0,97.0,97.0,4527,Jan Maijenbuurt,1,29359,0.00,78222.90,36.31,25461.52,39.48,1009.53,1.29,6.93,31.26,24451.99,18.84,4.49,14.35
3,1542.0,KK,KK04,Scheldebuurt Oost,K,202502-202503,512.0,497.0,5.0,0.0,502.0,97.0,98.0,4575,Scheldebuurt Oost,1,16775,0.00,140439.17,24.95,51886.30,51.37,5780.15,4.12,14.42,32.83,46106.16,28.17,6.13,22.04
5,549.0,EC,EC02,Houthavens Oost,E,202409-202410,49.0,47.0,0.0,0.0,47.0,96.0,96.0,4421,Houthavens Oost,1,28896,92.56,256234.18,1.42,14891.43,5.88,694.90,0.27,0.07,5.54,14196.53,0.40,0.27,0.13
6,1520.0,KF,KF01,Cornelis Troostbuurt,K,202502-202503,607.0,580.0,1.0,0.0,581.0,96.0,96.0,4467,Cornelis Troostbuurt,1,23871,6.25,156682.00,36.32,47809.58,37.21,618.15,0.39,6.70,30.12,47191.43,18.99,2.73,16.26
7,374.0,TE,TE03,G buurt Noord,T,202406,320.0,306.0,29.0,0.0,335.0,96.0,105.0,4799,G buurt Noord,1,22316,8.27,81241.17,10.73,44959.04,76.75,12430.82,15.30,21.41,40.04,32528.22,38.71,15.77,22.94
10,121.0,AA,AA05,Haarlemmerbuurt West,A,202403-202404,148.0,141.0,4.0,0.0,145.0,95.0,98.0,4303,Haarlemmerbuurt West,1,22537,7.52,118042.02,43.32,43872.29,40.67,247.52,0.21,3.50,36.96,43624.78,6.91,0.68,6.23
9,1521.0,KF,KF02,Lizzy Ansinghbuurt,K,202502-202503,239.0,226.0,2.0,0.0,228.0,95.0,95.0,4466,Lizzy Ansinghbuurt,1,19122,3.08,127548.67,40.61,33939.18,32.37,540.94,0.42,5.77,26.18,33398.24,17.84,2.51,15.33


In [48]:
gpdf03mp_d = gpdf03mp.sort_values(by=["Netto_PKD", "perc_public_green"], ascending=[True, False])
gpdf03mp_d.head(10)

,OBJECTNUMMER,Wijken,BRT_code,BRT_name,SDC,Periode,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
388,1272.0,MA,MA03,Oostelijke Handelskade,M,202501-202503,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4474,Oostelijke Handelskade,1,6381,39.35,364079.79,12.24,103670.81,29.06,7904.12,2.17,0.59,26.30,95766.69,3.07,2.22,0.85
387,1330.0,MN,MN09,Sportpark Voorland,M,202501-202503,204.0,2.0,0.0,0.0,2.0,1.0,1.0,4600,Sportpark Voorland,2,0,6.51,269864.58,0.01,72910.47,30.35,14060.37,5.21,3.33,21.81,58850.10,35.14,17.31,17.83
386,834.0,NQ,NQ02,Noorderstrook Oost,N,202410-202411,396.0,13.0,2.0,0.0,15.0,3.0,4.0,4680,Noorderstrook Oost,2,0,14.91,480031.74,0.46,118130.97,43.20,60712.97,12.65,18.59,11.96,57418.00,57.29,29.13,28.16
384,1567.0,KR,KR04,Amstelpark,K,202502-202503,405.0,17.0,1.0,0.0,18.0,4.0,4.0,4762,Amstelpark,2,24,21.59,622328.41,0.87,175408.23,72.63,73081.75,11.74,44.45,16.44,102326.48,64.02,12.97,51.05
383,391.0,TK,TK03,Gaasperplas,T,202406,224.0,8.0,0.0,0.0,8.0,4.0,4.0,4803,Gaasperplas,5,48,50.07,1193317.09,3.11,275916.74,38.07,133471.07,11.18,14.95,11.94,142445.67,31.81,13.82,17.99
385,1558.0,KQ,KQ01,Amsterdamse Bos,K,202502-202503,130.0,5.0,1.0,0.0,6.0,4.0,5.0,4754,Amsterdamse Bos,4,90,51.97,1111505.74,3.14,209079.26,40.77,120275.47,10.82,21.96,7.99,88803.79,37.67,11.47,26.20
382,1765.0,SA,SA01,Landelijk gebied Driemond,S,202503,429.0,20.0,0.0,0.0,20.0,5.0,5.0,4819,Landelijk gebied Driemond,4,90,35.27,816724.49,1.11,126680.91,33.40,81882.90,10.03,17.88,5.49,44798.01,50.69,21.55,29.14
381,494.0,BA,BA03,Westhaven Noord,B,202406,134.0,11.0,2.0,0.0,13.0,8.0,10.0,4404,Westhaven Noord,5,10,33.01,5696839.68,10.70,562995.71,10.47,104815.72,1.84,0.59,8.04,458179.99,6.39,4.23,2.16
380,1337.0,MQ,MQ06,Amstelkwartier Zuid,M,202501-202503,566.0,51.0,3.0,0.0,54.0,9.0,10.0,4612,Amstelkwartier Zuid,2,132,6.35,243707.66,30.83,80827.36,35.83,8146.50,3.34,2.67,29.82,72680.86,7.03,4.00,3.03
379,1315.0,ML,ML03,Rieteiland Oost,M,202501-202503,31.0,3.0,0.0,0.0,3.0,10.0,10.0,4570,Rieteiland Oost,3,3291,0.30,104224.37,11.75,48635.54,47.38,22436.61,21.53,0.71,25.14,26198.93,31.30,29.96,1.34


In [49]:
gpdf03mp.to_csv("slcd_mrgd_inner.csv", index=False)

In [35]:
#grouping by Wijken
gpdf03mp_avg1 = (
    gpdf03mp
    .groupby("Wijken", as_index=False)
    .agg({
        'SDC': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        'Periode': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        'Cap_open': 'mean',
        'Bez_open': 'mean',
        'Fout': 'mean',
        'Bez_tot': 'mean',
        'Netto_PKD': 'mean',
        'Bruto_PKD': 'mean',  
        'urbtn_indx_km2': 'mean',
        'pop_dnsty_km2': 'mean',
        'perc_water_total': 'mean',
        'm2_ngbhd': 'mean',
        'perc_built_envmt': 'mean',
        'm2_publc_space': 'mean',
        'perc_publc_space': 'mean',
        'm2_publc_green': 'mean',
        'perc_public_green': 'mean',
        'perc_publc_tree': 'mean',
        'perc_public_greennt': 'mean',
        'm2_publc_greennt': 'mean',
        'perc_green': 'mean',
        'perc_low_green': 'mean',
        'perc_tree': 'mean',
    })
)
gpdf03mp_avg1.head(10)

,Wijken,SDC,Periode,Cap_open,Bez_open,Fout,Bez_tot,Netto_PKD,Bruto_PKD,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
0,AA,A,202403-202404,137.000000,113.500000,3.000000,116.500000,82.333333,85.166667,1.0,16928.666667,26.361667,136403.250000,26.188333,33000.256667,35.648333,501.411667,0.598333,5.056667,29.993333,32498.846667,8.855000,1.183333,7.671667
1,AB,A,202403-202404,146.818182,121.727273,1.363636,123.090909,83.454545,84.818182,1.0,21205.090909,14.959091,86265.746364,37.590000,21854.310909,32.154545,252.490000,0.325455,4.143636,27.685455,21601.821818,11.160000,1.470909,9.689091
2,AC,A,202403-202404,230.250000,176.250000,0.250000,176.500000,79.750000,79.750000,1.0,14566.500000,20.785000,143219.637500,42.747500,23667.902500,21.170000,126.710000,0.087500,5.060000,16.022500,23541.192500,16.830000,1.625000,15.205000
3,AD,A,202403-202404,68.666667,47.333333,1.000000,48.333333,70.666667,72.000000,1.0,12972.666667,11.970000,62655.376667,42.506667,24208.340000,42.393333,57.156667,0.093333,3.723333,38.576667,24151.183333,4.563333,0.130000,4.433333
4,AE,A,202403-202404,24.250000,19.500000,2.000000,21.500000,73.250000,83.000000,1.0,10229.000000,13.507500,83331.715000,51.232500,21612.160000,28.427500,52.937500,0.057500,1.700000,26.670000,21559.222500,3.745000,0.305000,3.440000
5,AF,A,202403-202404,85.333333,60.000000,2.333333,62.333333,67.111111,69.888889,1.0,15434.222222,29.972222,119616.562222,28.834444,33004.117778,32.086667,507.691111,0.336667,3.983333,27.766667,32496.426667,7.973333,0.792222,7.181111
6,AG,A,202403-202404,184.666667,149.000000,0.000000,149.000000,80.000000,80.000000,1.0,12031.000000,19.246667,104181.230000,40.106667,17091.036667,22.350000,89.286667,0.076667,5.660000,16.613333,17001.750000,19.236667,2.390000,16.846667
7,AH,A,202403-202404,119.125000,97.500000,1.250000,98.750000,81.625000,82.750000,1.0,12483.750000,14.411250,81328.213750,39.570000,22924.062500,35.608750,636.821250,0.552500,7.613750,27.442500,22287.243750,13.065000,1.140000,11.925000
8,AJ,A,202403-202404,298.666667,231.000000,2.000000,233.000000,76.666667,77.000000,1.0,12117.666667,19.020000,182366.473333,27.300000,49554.150000,38.350000,2714.040000,2.830000,9.456667,26.063333,46840.106667,20.263333,3.976667,16.286667
9,AK,A,202403-202404,238.375000,187.500000,0.625000,188.125000,78.000000,78.000000,1.0,15583.375000,22.053750,164170.245000,23.428750,43390.987500,39.102500,4775.252500,3.805000,9.323750,25.973750,38615.733750,18.837500,5.375000,13.462500


In [36]:
gpdf03mp_avg1 = gpdf03mp_avg1.sort_values(by=["Netto_PKD", "perc_publc_space"], ascending=[False, True])
gpdf03mp_avg1.head(10)

,Wijken,SDC,Periode,Cap_open,Bez_open,Fout,Bez_tot,Netto_PKD,Bruto_PKD,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
48,KF,K,202502-202503,399.000000,374.750000,3.000000,377.75,94.250000,94.75,1.0,24135.250000,5.685000,136048.922500,39.620000,38602.830000,34.060000,497.910000,0.365000,5.722500,27.972500,38104.920000,16.015000,1.955,14.060000
92,TE,T,202406,720.666667,622.000000,48.000000,670.00,88.666667,97.00,1.0,12859.333333,6.400000,276551.660000,14.426667,145967.763333,69.010000,25601.780000,11.136667,15.096667,42.776667,120365.983333,28.816667,12.310,16.506667
24,EP,E,202409-202410,244.000000,213.500000,4.500000,218.00,87.000000,89.00,1.0,26797.000000,9.325000,136980.335000,41.045000,35596.490000,30.925000,1363.615000,1.175000,4.405000,25.345000,34232.875000,16.025000,2.935,13.090000
23,EN,E,202409-202410,205.000000,178.250000,0.500000,178.75,86.500000,86.75,1.0,24500.250000,8.765000,68965.350000,31.700000,19799.490000,32.170000,877.000000,1.545000,3.100000,27.525000,18922.490000,22.287500,5.460,16.827500
28,ET,E,202409-202410,608.500000,517.500000,2.000000,519.50,86.500000,86.50,1.0,25744.500000,3.955000,154075.040000,39.260000,47481.050000,35.230000,535.740000,0.315000,4.650000,30.265000,46945.315000,16.080000,2.585,13.495000
16,EF,E,202409-202410,587.000000,498.000000,2.500000,500.50,86.500000,86.50,1.0,20392.000000,7.975000,202632.165000,24.210000,66726.730000,51.395000,17273.600000,5.275000,17.120000,29.000000,49453.130000,32.515000,7.250,25.265000
94,TG,T,202406,601.500000,516.000000,12.000000,528.00,86.500000,88.50,1.0,14835.000000,10.460000,213014.830000,7.530000,111146.600000,79.360000,38361.775000,17.675000,27.385000,34.300000,72784.820000,46.300000,18.240,28.060000
25,EQ,E,202409-202410,737.000000,631.000000,5.000000,636.00,86.000000,86.00,1.0,21439.000000,16.360000,260010.150000,31.160000,75753.330000,38.490000,1904.890000,0.730000,9.360000,28.400000,73848.440000,20.120000,1.750,18.370000
29,EU,E,202409-202410,455.000000,384.666667,0.333333,385.00,84.666667,85.00,1.0,24713.666667,4.803333,116412.916667,37.310000,37008.990000,40.146667,1739.786667,1.076667,9.116667,29.953333,35269.203333,21.193333,2.470,18.723333
95,TH,T,202406,390.600000,333.000000,30.200000,363.20,84.400000,94.60,1.2,7763.400000,6.956000,192259.316000,13.158000,102024.264000,64.360000,14927.970000,8.114000,10.404000,45.842000,87096.294000,21.116000,9.322,11.794000


In [37]:
gpdf03mp_avg1 = gpdf03mp_avg1.sort_values(by=["Netto_PKD", "perc_publc_space"], ascending=[True, False])
gpdf03mp_avg1.head(10)

,Wijken,SDC,Periode,Cap_open,Bez_open,Fout,Bez_tot,Netto_PKD,Bruto_PKD,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
67,ML,M,202501-202503,31.000000,3.000000,0.000000,3.000000,10.000000,10.000000,3.000000,3291.000000,0.300000,1.042244e+05,11.750000,48635.540000,47.380000,22436.610000,21.530000,0.710000,25.140000,26198.930000,31.300000,29.960000,1.340000
36,FH,F,202409-202411,472.000000,63.000000,8.000000,71.000000,13.000000,15.000000,1.000000,991.000000,5.720000,1.179980e+05,36.560000,44197.100000,49.990000,3278.910000,2.780000,12.530000,34.680000,40918.190000,19.760000,3.170000,16.590000
10,BA,B,202406,105.500000,19.666667,1.333333,21.000000,17.833333,19.333333,5.000000,6.333333,23.721667,4.218151e+06,11.441667,599206.778333,16.013333,126789.336667,2.431667,1.278333,12.303333,472417.445000,11.815000,8.068333,3.746667
97,TK,T,202406,191.333333,34.000000,2.000000,36.000000,22.333333,24.000000,3.666667,1462.333333,26.320000,9.321291e+05,4.033333,279938.400000,59.713333,118258.636667,13.010000,28.433333,18.270000,161679.766667,48.606667,15.193333,33.413333
11,EA,E,202409-202410,287.000000,65.000000,0.000000,65.000000,23.000000,23.000000,1.000000,864.000000,8.200000,5.322527e+05,6.180000,72940.100000,26.690000,31278.950000,5.880000,12.980000,7.830000,41661.150000,65.880000,22.840000,43.040000
34,FF,F,202409-202411,391.666667,122.666667,3.666667,126.333333,29.000000,32.000000,2.666667,281.000000,17.716667,1.234561e+06,8.216667,363591.186667,36.773333,223546.116667,15.473333,10.540000,10.760000,140045.073333,48.296667,28.896667,19.400000
57,KQ,K,202502-202503,1058.400000,481.000000,5.000000,486.000000,29.200000,29.400000,1.800000,4073.600000,16.710000,5.609882e+05,15.488000,164436.202000,53.148000,45967.200000,7.264000,19.688000,26.196000,118469.000000,33.894000,9.368000,24.526000
55,KN,K,202502-202503,1096.000000,329.000000,1.000000,330.000000,30.000000,30.000000,1.000000,5489.000000,8.200000,2.674263e+05,18.550000,88641.760000,52.310000,8151.620000,3.050000,19.160000,30.100000,80490.140000,34.090000,7.380000,26.710000
58,KR,K,202502-202503,1254.250000,483.750000,4.250000,488.000000,31.750000,32.000000,1.250000,6011.750000,10.117500,4.258804e+05,13.147500,146423.682500,65.130000,37358.492500,8.010000,29.347500,27.772500,109065.192500,43.085000,9.365000,33.720000
56,KP,K,202502-202503,149.750000,36.750000,2.250000,39.000000,35.250000,36.500000,1.250000,1061.000000,5.122500,4.891398e+05,20.722500,175161.982500,39.230000,16870.860000,2.775000,5.847500,30.607500,158291.122500,14.967500,5.817500,9.150000


In [44]:
#in case we select the buurt out of only the parking pressure database
#top results
gpdf01_2.head()

,OBJECTNUMMER,Wijken,BRT_code,BRT_name,SDC,Periode,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD
157,563.0,EG,EG02,GWL terrein,E,202409-202410,38.0,39.0,0.0,0.0,39.0,103.0,103.0
70,357.0,TA,TA03,Amstel III deel A/B Noord,T,202406,4.0,4.0,0.0,0.0,4.0,100.0,100.0
89,376.0,TF,TF02,Grunder/Koningshoef,T,202406,425.0,416.0,24.0,0.0,440.0,98.0,104.0
171,577.0,EK,EK03,Jan Maijenbuurt,E,202409-202410,260.0,251.0,0.0,0.0,251.0,97.0,97.0
466,1542.0,KK,KK04,Scheldebuurt Oost,K,202502-202503,512.0,497.0,5.0,0.0,502.0,97.0,98.0


In [46]:
#in case we select the buurt out of only the parking pressure database
#bottom results
gpdf01_2d = gpdf01_2.sort_values(by=['Netto_PKD'], ascending=[True])
gpdf01_2d.head()

,OBJECTNUMMER,Wijken,BRT_code,BRT_name,SDC,Periode,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD
343,1272.0,MA,MA03,Oostelijke Handelskade,M,202501-202503,9.0,0.0,0.0,0.0,0.0,0.0,0.0
373,1302.0,MH,MH03,Oostpunt Zeeburgereiland,M,202501-202503,97.0,1.0,0.0,0.0,1.0,1.0,1.0
401,1330.0,MN,MN09,Sportpark Voorland,M,202501-202503,204.0,2.0,0.0,0.0,2.0,1.0,1.0
382,1311.0,MK,MK02,Pampusbuurt West,M,202501-202503,229.0,2.0,0.0,0.0,2.0,1.0,1.0
123,500.0,BB,BB03,Minervahaven Noordoost,B,202406,62.0,2.0,0.0,0.0,2.0,3.0,3.0
